In [ ]:
from requests_oauthlib import OAuth1Session

In [ ]:
import json

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
%matplotlib inline

In [ ]:
KEYS = {
    'consumer_key' : '<your consumer_key>',
    'consumer_secret' : '<your consumer_secret>',
    'access_token' : 'your access_token',
    'access_secret' : 'your access_secret',
}

In [ ]:
twitter = OAuth1Session(KEYS['consumer_key'],KEYS['consumer_secret'],KEYS['access_token'],KEYS['access_secret'])

In [ ]:
def getTwitterData(key_word, repeat):
    url = "https://api.twitter.com/1.1/search/tweets.json"
    params ={'q': key_word, 'count':'100','lang':'ja', 'result_type':'recent'}
    tweets = []
    mid = -1
    break_flag = 0
    
    for i in range(repeat):
        params['max_id'] = mid
        res = twitter.get(url, params = params)
        if res.status_code == 200:
            sub_tweets = json.loads(res.text)['statuses']
            limit = res.headers['x-rate-limit-remaining'] if 'x-rate-limit-remaining' in res.headers else 0
#            print("API残接続可能回数：%s" % len(limit))            
            tweet_ids = []
            for tweet in sub_tweets:
                tweet_ids.append(int(tweet['id']))
                tweets.append(tweet)
            if len(tweet_ids) > 0:
                min_tweet_id = min(tweet_ids)
                mid = min_tweet_id - 1
            else:
                break_flag = 1
                break;
                
            ## 終了判定
            if break_flag == 1:
                break;
                
        else:
            print("Failed: %d" % res.status_code)
            break_flag = 1
    
    print("ツイート取得数：%s" % len(tweets))
        
    return tweets

In [ ]:
tweets = getTwitterData("<key word> <-RT:リツイートを取り除く>", 180)

In [ ]:
df_tweet = pd.DataFrame(tweets)

In [ ]:
df_tweet.to_csv("<フィール名>.csv")

In [ ]:
df_tweet["created_at"] = pd.to_datetime(df_tweet["created_at"])

In [ ]:
plt.figure(figsize=(20,5))
g = sns.barplot(x="created_at", y="id", data=df_tweet.set_index("created_at").resample("h")["id"].count().reset_index())
labels = g.get_xticklabels()
g.set_xticklabels(labels, rotation=90)
plt.show()